In [36]:
import numpy as np
from scipy.optimize import*
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from tkinter import*
from tkinter import ttk
import pandas as pd
def fractional_flow():
    Fw_root = Tk()
    Fw_root.iconbitmap('STRATV.ico')
    Fw_root.title('STRAT-V')
    Frame1 = LabelFrame(Fw_root, text='Result').place(relheight=0.3, relwidth=0.55,rely=0, relx=0.05)
    listbox = Listbox(Frame1)
    listbox.place(relheight=0.25, relwidth=0.55,rely=0.1, relx=0.05)

    # Creating a Treeview
    Frame2 = LabelFrame(Fw_root, text='Fractional Flow Data').place(relheight=0.5, relwidth=0.8,rely=0.5, relx=0.05)
    tree = ttk.Treeview(Frame2)
    tree.place(relheight=0.4, relwidth=0.8,rely=0.6, relx=0.05)

    treescrolly = Scrollbar(Frame2, orient='vertical',command=tree.yview)
    treescrollx = Scrollbar(Frame2, orient='horizontal',command=tree.xview)
    tree.configure(xscrollcommand=treescrollx.set)
    tree.configure(yscrollcommand=treescrolly.set)
    treescrollx.pack(side='bottom', fill='x')
    treescrolly.pack(side='right', fill='y')

    # Water Saturation
    SW = np.array([0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65])
    SW_table = pd.DataFrame(SW, columns = ['SW'])
    #print(SW_table)
    # Relative permeability of Water
    KRW = np.array([0, 0.003, 0.008, 0.018, 0.035, 0.054, 0.08, 0.105, 0.14, 0.18])

    # Relative permeability of Oil
    KRO = np.array([1, 0.68, 0.46, 0.32, 0.2, 0.124, 0.071, 0.038, 0.017, 0])

    # Water and Oil Viscosity
    VISW = 0.95 # unit is in centipoise
    VISO = 3.6  # unit is in centipoise

    # Initial water saturation
    SWI = 0.2

    # Using the correlation between relative permeability ratio and water saturation
    #print('Correlation:\n Kro/Krw = aexp(-bSw)\n')

    # Calculating the coefficient b
    b = (np.log((KRO/KRW)[2])-np.log((KRO/KRW)[3]))/(SW[3]-SW[2])
    #print('b is:\n ', b)
    #========================================================================

    # Calculating the coefficient a
    a = (KRO/KRW)[2]*math.exp(b*SW[2])
    #print('a is:\n ', a)
    #========================================================================
    # Calculating the fractional flow
    def fw(SW):
        fw = 1/(1+a*(VISW/VISO)*np.exp(-b*SW))
        return(fw)
    #========================================================================
    '''' To calculate a suitable slope for the tangent to the fractional flow curve
    Drawn from the initial water saturation'''

    # STEP1: Generate a list of uniformly distributed random numbers from a water saturation
    # greater than the initial water saturation to 1
    xList = []
    for i in range(0, 10000):
        x = random.uniform(SWI+0.1, 1)
        xList.append(x) 
    xs = np.array(xList)

    # STEP2: Calculate different slopes of tangents or lines intersecting the fractional
    # flow curve using the array generated in step 1 as the water saturation.
    m = 1/((xs-SWI)*(1+(VISW/VISO)*a*np.exp(-b*xs)))

    # STEP3: Calculate the maximum slope from different slopes generated in step 2.
    # The value of this slope will be the slope of the tangent to the fractional flow
    # curve.
    tangent_slope=max(m)
    #print('slope of the tangent line is:\n ',tangent_slope)
    #==========================================================================
    # Calculate the breakthrough saturation.
    Saturation_at_Breakthrough = SWI + 1/tangent_slope
    #print('saturation at breakthrough is:\n ', Saturation_at_Breakthrough)
    #===========================================================================
    # Calculating the saturation at the flood front

    def funct(SWF):
        swf = SWF[0]
        F = np.empty((1))
        F[0] = ((tangent_slope*(swf-SWI)*(1+(VISW/VISO)*a*math.exp(-b*swf)))-1)
        return F
    SWF_Guess = np.array([SWI+0.1])
    SWF = fsolve(funct, SWF_Guess)[0]
    SWF
    #============================================================================
    # Fractional flow at the flood front
    Fwf = fw(SWF)
    Fwf
    #=============================================================================
    # Fractional flow
    Fw = fw(SW)
    Fw_table = pd.DataFrame(Fw, columns = ['Fractional Flow (Fw)'])
    #print(Fw_table)
    #=============================================================================
    # Calculating the differential of the fractional flow equation
    dfw_dSw = (VISW/VISO)*a*b*np.exp(-SW*b)/(1+(VISW/VISO)*a*np.exp(-SW*b))**2
    dfw_dSw_table = pd.DataFrame(dfw_dSw, columns = ['dFw/dSw'])
    #print(dfw_dSw_table)
    #============================================================================
    # Generating the data for the tangent plot
    tangent = (SW-SWI)*tangent_slope
    tangent_table = pd.DataFrame(tangent, columns = ['Tangent'])
    #print(tangent_table)
    #============================================================================
    Fractional_flow_table = pd.concat([SW_table, Fw_table, dfw_dSw_table, tangent_table], axis=1)
    #print(Fractional_flow_table)
    #=============================================================================
    # Making the plots
    %matplotlib
    fig, ax = plt.subplots(constrained_layout=True)
    fig.set_figheight(4)
    fig.set_figwidth(7)
    fractional_flow_curve = ax.plot(SW, fw(SW), 'b', label = 'Fractional Flow (Fw)')
    tangent_curve = ax.plot(SW, tangent, 'k--')
    ax.set_ylabel("Fractional Flow (fw)",fontsize=14)
    ax.set_xlabel("Water Saturation (Sw)",fontsize=14)
    ax.set_ylim([0,1])
    ax.set_xlim([0,1])
    # twin object for two different y-axis on the same plot
    ax2=ax.twinx()
    # make a plot with different y-axis using second axis object
    dfw_dSw_curve = ax2.plot(SW, dfw_dSw, 'r', label ='dFw/dSw')
    ax2.set_ylabel("dfw/dSw",fontsize=14)
    ax.grid(True)
    ax2.legend()
    ax.legend(loc='upper left')
    ax.annotate("  (Swf, Fwf)", (SWF, Fwf))
    ax.annotate(" SwBT", (Saturation_at_Breakthrough, 1))
    plt.show()
    plt.ion()
    #=========================================================================================================
    listbox.insert(1,"  Correlation: Kro/Krw = aexp(-bSw)") 
    listbox.insert(2, '  b : ' +str(b)) 
    listbox.insert(3, '  a : '+ str(a))
    listbox.insert(4, '  Slope of the tangent line : ' + str(tangent_slope)) 
    listbox.insert(5, '  Flood Front Saturation (Swf) : '+ str(SWF))
    listbox.insert(6, '  Flood Front Fractional Flow (Fwf) : '+ str(Fwf))
    listbox.insert(7, '  Saturation at breakthrough (SwBT) : '+ str(Saturation_at_Breakthrough)) 

    tree["column"]=list(Fractional_flow_table.columns)
    tree['show']='headings'
    for column in tree['columns']:
        tree.heading(column,text=column)
    Fractional_flow_table_rows = Fractional_flow_table.to_numpy().tolist()
    for row in Fractional_flow_table_rows:
        tree.insert("","end", values=row)
        #return None


    Fw_root.geometry("600x500")

    Fw_root.mainloop() 


<ipython-input-36-ca17d90bc2b2>:51: RuntimeWarning: divide by zero encountered in true_divide
  b = (np.log((KRO/KRW)[2])-np.log((KRO/KRW)[3]))/(SW[3]-SW[2])
<ipython-input-36-ca17d90bc2b2>:56: RuntimeWarning: divide by zero encountered in true_divide
  a = (KRO/KRW)[2]*math.exp(b*SW[2])


Using matplotlib backend: Qt5Agg
